In [1]:
import os
from env.audit import Audits

appsecret = os.getenv("admin_appsecret")
appid = os.getenv("admin_appid")
tenantid = os.getenv("admin_tenantid")

audit = Audits()
audit.set_ServicePrincipal(
    tenant_id=tenantid,
    client_id=appid,
    client_secret= appsecret
)

# audit.set_ApplicationModules("Apps")
# audit.set_ApplicationModules("Activity,Apps,Capacity,Catalog,Domains,FabricItems,Gateway,Graph,Refreshables,RefreshHistory,Roles,Tenant,Workspaces")
audit.set_ApplicationModules("Activity,Apps")
# audit.set_ApplicationModules("Roles,Tenant,Workspaces")

# a list of the supported logging levels
import logging
Logging_levels = {
            "DEBUG": logging.DEBUG,
            "INFO": logging.INFO,
            "WARNING": logging.WARNING,
            "ERROR": logging.ERROR,
            "CRITICAL": logging.CRITICAL
}

audit.set_LakehouseName("FabricLake")
audit.set_PathInLakehouse("stage")
audit.set_WorkspaceName("FabricMonitor")
audit.set_logging_config("INFO", "brandon.log")

audit.set_Activity_cron("* * * * * 30")
audit.set_Catalog_cron("* * * * * 30")
audit.set_Capacity_cron("* * * * * 30")
audit.set_Apps_cron("* * * * * 30")
audit.set_Domains_cron("* * * * * 30") # Not yet implemented
audit.set_Graph_cron("* * * * * 30")
audit.set_Tenant_cron("* * * * * 30")
audit.set_RefreshHistory_cron("* * * * * 30")
audit.set_Refreshables_cron("* * * * * 30")
audit.set_Gateway_cron("* * * * * 30")

# print(audit.ServicePrincipal)
await audit.run()



Task Activity is now running
2024-06-25 15:25:38,288 - env.context - WARNING - Activity is now running...
2024-06-25 15:25:38,290 - env.context - ERROR - Activity error...
2024-06-25 15:25:38,292 - env.context - CRITICAL - Activity critical...
Task Catalog Snapshots (Published Apps) is now running
Task Apps elapsed time: 2.0
Task Activity elapsed time: 10.6

Total elapsed time: 10.6


In [ ]:
d = {
    "name": "brandon",
    "age": 54
}

if hasattr(d, "name"):
    print("yes")
else:
    print("no")

